## Map Mutations to Protein-Ligand Interactions
Here we find and visualize the mutations at protein-ligand binding sites.

In [1]:
# Disable Numba: temporary workaround for https://github.com/sbl-sdsc/mmtf-pyspark/issues/288
import os
os.environ['NUMBA_DISABLE_JIT'] = "1"

In [2]:
from pyspark.sql import SparkSession
from mmtfPyspark.io import mmtfReader
from mmtfPyspark.interactions import InteractionExtractor, InteractionFilter
from ipywidgets import interact, IntSlider, FloatSlider
import py3Dmol
import pandas as pd

In [3]:
# Initialize Spark
spark = SparkSession.builder.appName("3-MapLigandInteractions").getOrCreate()

In [4]:
# Enable Arrow-based columnar data transfers between Spark and Pandas dataframes
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

#### Input parameters

In [5]:
distance_cutoff = 8  # distance cutoff for finding and visualizing interactions
input_file_name = 'mutations3d.csv'  # mutations mapped to 3D protein structures
output_file_name = 'mutations3d_ligand.csv'  # mutations mapped to protein-ligand binding sites

### Read 'mutations3d.csv' file created in the previous step

In [6]:
pd.set_option('display.max_columns', None)  # show all columns
df = pd.read_csv(input_file_name)
df['pdbPosition'] = df['pdbPosition'].astype('str')  # PDB residue numbers must be string to handle insertion codes
df.head()

,coverage,description,name,resolution,structureChainId,structureId,uniprotPosition,pdbPosition,residue,variationId,annotation,scale,color
0,0.026709,"Spike protein S2', water",Spike glycoprotein,2.47,6LVN.A,6LVN,1169,2,1169,"S:p.1169I>L(54), S:p.1169I>F(28), S:p.1169I>V(...","1169I>L(54), 1169I>F(28), 1169I>V(16), 1169I>T(6)",0.304365,#fcaa8d
1,0.027494,"Spike protein S2', water",Spike glycoprotein,2.47,6LVN.B,6LVN,1169,2,1169,"S:p.1169I>L(54), S:p.1169I>F(28), S:p.1169I>V(...","1169I>L(54), 1169I>F(28), 1169I>V(16), 1169I>T(6)",0.304365,#fcaa8d
2,0.026709,"Spike protein S2', water",Spike glycoprotein,2.47,6LVN.D,6LVN,1169,2,1169,"S:p.1169I>L(54), S:p.1169I>F(28), S:p.1169I>V(...","1169I>L(54), 1169I>F(28), 1169I>V(16), 1169I>T(6)",0.304365,#fcaa8d
3,0.273370,"Spike glycoprotein, 2-acetamido-2-deoxy-beta-D...",Spike glycoprotein,3.00,6XRA.A,6XRA,1169,1169,1169,"S:p.1169I>L(54), S:p.1169I>F(28), S:p.1169I>V(...","1169I>L(54), 1169I>F(28), 1169I>V(16), 1169I>T(6)",0.304365,#fcaa8d
4,0.273370,"Spike glycoprotein, 2-acetamido-2-deoxy-beta-D...",Spike glycoprotein,3.00,6XRA.B,6XRA,1169,1169,1169,"S:p.1169I>L(54), S:p.1169I>F(28), S:p.1169I>V(...","1169I>L(54), 1169I>F(28), 1169I>V(16), 1169I>T(6)",0.304365,#fcaa8d


## Create a list of unique PDB IDs

In [7]:
pdb_ids = list(df.structureId.drop_duplicates())
print("PDB Ids:", pdb_ids)

PDB Ids: ['6LVN', '6XRA', '6LXT', '6VSB', '6VXX', '6VYB', '6WPS', '6WPT', '6X29', '6X2A', '6X2B', '6X2C', '6X6P', '6X79', '6XCM', '6XCN', '6XEY', '6XF5', '6XF6', '6XKL', '6XLU', '6XM0', '6XM3', '6XM4', '6XM5', '6XR8', '6XS6', '6Z43', '6Z97', '6ZB4', '6ZB5', '6ZDH', '6ZGE', '6ZGG', '6ZGH', '6ZGI', '6ZHD', '6ZOW', '6ZOX', '6ZOY', '6ZOZ', '6ZP0', '6ZP1', '6ZP2', '6ZP5', '6ZP7', '6ZWV', '6ZXN', '7A25', '7A29', '7A4N', '7A93', '7A94', '7A95', '7A96', '7A97', '7A98', '7AD1', '7BNM', '7BNN', '7BNO', '7BYR', '7C2L', '7CAB', '7CAC', '7CAI', '7CAK', '7CHH', '7CN9', '7CT5', '7CWL', '7CWM', '7CWN', '7CWS', '7CWU', '7CZP', '7CZQ', '7CZR', '7CZS', '7CZT', '7CZU', '7CZV', '7CZW', '7CZX', '7CZY', '7CZZ', '7D00', '7D03', '7D0B', '7D0C', '7D0D', '7DCC', '7DCX', '7DD2', '7DD8', '7DDD', '7DDN', '7DF3', '7DF4', '7DK3', '7DK4', '7DK5', '7DK6', '7DK7', '7DWX', '7DWY', '7DWZ', '7DX0', '7DX1', '7DX2', '7DX3', '7DX5', '7DX6', '7DX7', '7DX8', '7DX9', '7E7B', '7E7D', '7JJI', '7JJJ', '7JV4', '7JV6', '7JVC', '7JW0'

## Find all polymer-ligand interactions

In [8]:
structures = mmtfReader.download_mmtf_files(pdb_ids)

In [9]:
interaction_filter = InteractionFilter(distanceCutoff=distance_cutoff)
interaction_filter.set_query_groups(False, ["HOH", "DOD"])  # exclude water interactions

interactions = InteractionExtractor().get_ligand_polymer_interactions(structures, interaction_filter, level='group').toPandas()
interactions.head()

,structureChainId,queryGroupId,queryChainId,queryGroupNumber,targetGroupId,targetChainId,targetGroupNumber,sequenceIndex,sequence
0,6XC2.A,NAG,A,601,ALA,A,372,53,RVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVAD...
1,6XC2.Z,NAG,Z,601,SER,Z,371,52,RVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVAD...
2,6XC2.Z,NAG,Z,601,ASN,Z,343,24,RVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVAD...
3,6XC2.A,NAG,A,601,ASP,A,364,45,RVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVAD...
4,6XC2.Z,NAG,Z,601,ALA,Z,372,53,RVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVAD...


## Filter mutations by joining with the interaction data

In [10]:
mt = df.merge(interactions, left_on=['structureChainId','pdbPosition'], right_on=['structureChainId','targetGroupNumber'], how='inner')
mt.head()

,coverage,description,name,resolution,structureChainId,structureId,uniprotPosition,pdbPosition,residue,variationId,annotation,scale,color,queryGroupId,queryChainId,queryGroupNumber,targetGroupId,targetChainId,targetGroupNumber,sequenceIndex,sequence
0,0.27337,"Spike glycoprotein, 2-acetamido-2-deoxy-beta-D...",Spike glycoprotein,3.0,6XRA.A,6XRA,1170,1170,1170,"S:p.1170S>F(26), S:p.1170S>T(14), S:p.1170S>A(...","1170S>F(26), 1170S>T(14), 1170S>A(6), 1170S>P(...",0.258940,#fcb79c,NAG,A,1417,SER,A,1170,1169,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
1,0.27337,"Spike glycoprotein, 2-acetamido-2-deoxy-beta-D...",Spike glycoprotein,3.0,6XRA.B,6XRA,1170,1170,1170,"S:p.1170S>F(26), S:p.1170S>T(14), S:p.1170S>A(...","1170S>F(26), 1170S>T(14), 1170S>A(6), 1170S>P(...",0.258940,#fcb79c,NAG,B,1417,SER,B,1170,1169,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
2,0.27337,"Spike glycoprotein, 2-acetamido-2-deoxy-beta-D...",Spike glycoprotein,3.0,6XRA.C,6XRA,1170,1170,1170,"S:p.1170S>F(26), S:p.1170S>T(14), S:p.1170S>A(...","1170S>F(26), 1170S>T(14), 1170S>A(6), 1170S>P(...",0.258940,#fcb79c,NAG,C,1417,SER,C,1170,1169,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
3,0.27337,"Spike glycoprotein, 2-acetamido-2-deoxy-beta-D...",Spike glycoprotein,3.0,6XRA.A,6XRA,1171,1171,1171,"S:p.1171G>S(94), S:p.1171G>V(54), S:p.1171G>D(34)","1171G>S(94), 1171G>V(54), 1171G>D(34)",0.341039,#fc9d7f,NAG,A,1417,GLY,A,1171,1170,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
4,0.27337,"Spike glycoprotein, 2-acetamido-2-deoxy-beta-D...",Spike glycoprotein,3.0,6XRA.B,6XRA,1171,1171,1171,"S:p.1171G>S(94), S:p.1171G>V(54), S:p.1171G>D(34)","1171G>S(94), 1171G>V(54), 1171G>D(34)",0.341039,#fc9d7f,NAG,B,1417,GLY,B,1171,1170,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...


## Save protein-ligand mapping

In [11]:
mt.to_csv(output_file_name, index=False)

## View mutations grouped by protein chain
Use the slider to view each protein chain. Turn labels off for an unobstructed view of the mutations. Interacting ligands are rendered as spheres with green carbon atoms.

In [12]:
chains = mt.groupby('structureChainId')
print("Chains:", list(chains.groups.keys()))

Chains: ['6LXT.A', '6LXT.B', '6LXT.C', '6LXT.E', '6LXT.F', '6LZG.B', '6M0J.E', '6M17.E', '6M17.F', '6VSB.A', '6VSB.B', '6VSB.C', '6VW1.E', '6VW1.F', '6VXX.A', '6VXX.B', '6VXX.C', '6VYB.A', '6VYB.B', '6VYB.C', '6W41.C', '6WPS.A', '6WPS.B', '6WPS.E', '6WPT.A', '6WPT.B', '6WPT.C', '6X6P.A', '6X6P.B', '6X6P.C', '6X79.A', '6X79.B', '6X79.C', '6XC2.A', '6XC2.Z', '6XC3.C', '6XC4.A', '6XC4.Z', '6XC7.A', '6XCM.A', '6XCM.B', '6XCM.C', '6XCN.A', '6XCN.C', '6XCN.E', '6XDG.E', '6XE1.E', '6XEY.A', '6XEY.B', '6XEY.C', '6XF5.A', '6XF5.B', '6XF5.C', '6XF6.A', '6XF6.B', '6XF6.C', '6XKL.A', '6XKL.B', '6XKL.C', '6XKP.A', '6XKP.B', '6XKQ.A', '6XLU.A', '6XLU.B', '6XLU.C', '6XM0.A', '6XM0.B', '6XM0.C', '6XM3.A', '6XM3.B', '6XM3.C', '6XM4.A', '6XM4.B', '6XM4.C', '6XM5.A', '6XM5.B', '6XM5.C', '6XR8.A', '6XR8.B', '6XR8.C', '6XRA.A', '6XRA.B', '6XRA.C', '6YLA.A', '6YLA.E', '6YZ5.E', '6YZ7.AAA', '6YZ7.EEE', '6Z2M.A', '6Z2M.E', '6Z43.A', '6Z43.B', '6Z43.C', '6Z97.A', '6Z97.B', '6Z97.C', '6ZB4.A', '6ZB4.B', '6ZB4.C

In [13]:
# Setup viewer
def view_grouped_mutations(grouped_df, *args):
    chainIds = list(grouped_df.groups.keys())

    def view3d(show_bio_assembly, show_surface, show_short_label, show_long_label, size, font, logFreq, i):
        group = grouped_df.get_group(chainIds[i])
        
        pdb_id, chain_id = chainIds[i].split('.')
        global viewer1
        viewer1 = py3Dmol.view(query='pdb:' + pdb_id, options={'doAssembly': show_bio_assembly}, width=size, height=size)

        # polymer style
        viewer1.setStyle({'cartoon': {'colorscheme': 'chain', 'width': 0.6, 'opacity':0.9}})

        # highlight chain of interest in blue
        viewer1.setStyle({'chain': chain_id},{'cartoon': {'color': 'blue'}})
   
        # non-polymer style
        viewer1.setStyle({'hetflag': True}, {'stick':{'radius': 0.3, 'singleBond': False}})
        
        # don't display water molecules
        viewer1.setStyle({'resn': ['HOH','DOD']}, {})
        
        rows = group.shape[0]
        for j in range(0, rows):
            if group.iloc[j]['scale'] > logFreq:
                # interacting residue info
                res_num = str(group.iloc[j]['pdbPosition'])
                mod_res = {'resi': res_num, 'chain': chain_id}
            
                # interacting ligand info
                lig_id = group.iloc[j]['queryGroupId']
                lig_chain = group.iloc[j]['queryChainId']
                lig_num = group.iloc[j]['queryGroupNumber']
                lig_res = {'resi': lig_num, 'chain': lig_chain}
            
                col = group.iloc[j]['color']
                c_col = col + 'Carbon'
                viewer1.addStyle(mod_res, {'stick':{'colorscheme':c_col, 'radius': 0.2}})
                viewer1.addStyle(mod_res, {'sphere':{'color':col, 'opacity': 0.6}})   
                viewer1.addStyle(lig_res, {'sphere': {'colorscheme': 'greenCarbon'}})
            
                annotation = group.iloc[j]['annotation']
                variationId = group.iloc[j]['variationId']
                
                if show_short_label:
                    label = annotation
                if show_long_label:
                    label = chain_id + "." + str(res_num) + ": " + variationId
                if show_short_label or show_long_label:
                    viewer1.addLabel(label, {'fontSize':font,'fontColor': 'black','backgroundColor':'ivory'}, mod_res)
                    viewer1.addLabel(lig_id + lig_num, {'fontSize':font}, lig_res)

        viewer1.zoomTo({'chain': chain_id})
        viewer1.center({'chain': chain_id})
        
        if show_surface:
             viewer1.addSurface(py3Dmol.SES,{'opacity':0.8,'color':'lightblue'},{'chain': chain_id})
                
        # print header
        name = group.iloc[0]['name']
        resolution = group.iloc[0]['resolution']
        coverage = group.iloc[0]['coverage']
        description = group.iloc[0]['description']
        print(name)
        print()
        print(f'PDB Id: {pdb_id}, chain Id: {chain_id}, resolution: {resolution}, sequence coverage: {coverage:.2f}')
        print(f'description: {description}')
        
        # print any specified additional columns from the dataframe
        for a in args:
            print(a + ": " + group.iloc[j][a])
                
        return viewer1.show()

    f_widget = IntSlider(value=9, min=5, max=20, description='font size', continuous_update=False)
    z_widget = IntSlider(value=750, min=500, max=1200, description='size', continuous_update=False)
    s_widget = IntSlider(min=0, max=len(chainIds)-1, description='structure', continuous_update=False)
    l_widget = FloatSlider(value=0.5, min=0, max=1, step=0.05, description='logFreq:', disabled=False, 
                           continuous_update=False, orientation='horizontal', readout=True, readout_format='.2f')
    
    return interact(view3d, show_bio_assembly=False, show_surface=False, show_short_label=True, show_long_label=False, size=z_widget, 
                    font=f_widget, logFreq=l_widget, i=s_widget)

def view_image1():
    return viewer1.png()

In [14]:
view_grouped_mutations(chains);

interactive(children=(Checkbox(value=False, description='show_bio_assembly'), Checkbox(value=False, descriptio…

## View one mutation at a time
Use the slider to view each mutation. Surrounding residues within the `distance_cutoff` are rendered as orange sticks. Interacting ligands are rendered as spheres with green carbon atoms.

In [15]:
# Setup viewer
def view_single_mutation(df, distance_cutoff, *args):

    def view3d(show_bio_assembly, show_surface, show_short_label, show_long_label, size, font, logFreq, i):        
        pdb_id, chain_id = df.iloc[i]['structureChainId'].split('.')
        
        global viewer2
        viewer2 = py3Dmol.view(query='pdb:' + pdb_id, options={'doAssembly': show_bio_assembly}, width=size, height=size)

        # polymer style
        viewer2.setStyle({'cartoon': {'colorscheme': 'chain', 'width': 0.6, 'opacity':0.7}})
       
        # highlight chain of interest in blue
        viewer2.setStyle({'chain': chain_id},{'cartoon': {'color': 'blue', 'width': 0.6, 'opacity':0.5}})
        
        # non-polymer style
        viewer2.setStyle({'hetflag': True}, {'stick':{'radius': 0.3, 'singleBond': False}})
        
        # don't display water molecules
        viewer2.setStyle({'resn': ['HOH','DOD']}, {})
        
        # interacting residue info
        res_num = str(df.iloc[i]['pdbPosition'])
        mod_res = {'resi': res_num, 'chain': chain_id}
        
        # interacting ligand info
        label = df.iloc[i]['variationId']  
        lig_id = df.iloc[i]['queryGroupId']
        lig_chain = df.iloc[i]['queryChainId']
        lig_num = df.iloc[i]['queryGroupNumber']
        lig_res = {'resi': lig_num, 'chain': lig_chain}
        lig_label = lig_id + "-" + lig_chain + lig_num
        
        col = df.iloc[i]['color']
        c_col = col + 'Carbon'
        viewer2.addStyle(mod_res, {'stick':{'colorscheme': c_col, 'radius': 0.2}})
        viewer2.addStyle(mod_res, {'sphere':{'color': col, 'opacity': 0.8}})  

        # text label
        annotation = df.iloc[i]['annotation']
        variationId = df.iloc[i]['variationId']
                
        if show_short_label:
            label = annotation
        if show_long_label:
            label = chain_id + "." + str(res_num) + ": " + variationId
        if show_short_label or show_long_label:
            viewer2.addLabel(label, {'fontSize':font,'fontColor': 'black','backgroundColor':'ivory', 'opacity': 1.0}, mod_res)    
            viewer2.addLabel(lig_label, {'fontSize':font-2}, lig_res)      

        # select neigboring residues by distance
        surroundings = {'chain': chain_id, 'resi': res_num, 'byres': True, 'expand': distance_cutoff}
        
        # residues surrounding mutation site
        viewer2.addStyle(surroundings,{'stick':{'colorscheme':'orangeCarbon', 'radius': 0.15}})

        
        # interacting ligand style
        viewer2.addStyle(lig_res, {'sphere': {'colorscheme': 'greenCarbon'}})  
        
        # set style for interacting waters
        waters = {'resn': ['HOH','DOD']}
        waters.update(surroundings)
        viewer2.addStyle(waters,{'sphere':{'color':'orange', 'radius': 0.5}})
    
        if show_surface:
             viewer2.addSurface(py3Dmol.SES, {'opacity':0.8,'color':'lightblue'}, {'chain': chain_id})
         
        viewer2.zoomTo(surroundings)
        viewer2.center(mod_res)
        
        # print header
        print("PDB Id:", pdb_id, "chain Id:" , chain_id, "residue:", res_num, "ligand:", lig_label, "mutation:", variationId)
        
        # print any specified additional columns from the dataframe
        for a in args:
            print(a + ": " + df.iloc[i][a])
                
        return viewer2.show()

    f_widget = IntSlider(value=12, min=5, max=20, description='font size', continuous_update=False)
    z_widget = IntSlider(value=750, min=500, max=1200, description='size', continuous_update=False)
    s_widget = IntSlider(min=0, max=len(df)-1, description='structure', continuous_update=False)
    l_widget = FloatSlider(value=0.5, min=0, max=1, step=0.05, description='logFreq:', disabled=False, 
                           continuous_update=False, orientation='horizontal', readout=True, readout_format='.2f')
    
    return interact(view3d, show_bio_assembly=False, show_surface=False, show_short_label=True, show_long_label=False, size=z_widget, 
                    font=f_widget, logFreq=l_widget, i=s_widget)

def view_image2():
    return viewer2.png()

In [16]:
mt_unique = mt.drop_duplicates(["structureChainId","variationId",'queryGroupId','queryGroupNumber'])

In [17]:
view_single_mutation(mt_unique, distance_cutoff);

interactive(children=(Checkbox(value=False, description='show_bio_assembly'), Checkbox(value=False, descriptio…

In [18]:
# Shutdown Spark
spark.stop()

## Now run the next step
Map mutations occuring at protein-polymer interfaces: [4-MapToDrugInteractions.ipynb](4-MapToDrugInteractions.ipynb)